In [ ]:
import pandas as pd         # 导入pandas库，它就像一个超级强大的Excel工具，专门用来处理表格数据（DataFrame）。
from sklearn.model_selection import train_test_split # 从sklearn.model_selection模块导入train_test_split函数，用于将数据集划分为训练集和测试集。
from sklearn.linear_model import LinearRegression # 从sklearn.linear_model模块导入LinearRegression类，这是我们将要使用的线性回归模型。
from sklearn.metrics import mean_squared_error, r2_score # 从sklearn.metrics模块导入均方误差（MSE）和决定系数（R²）这两个评估回归模型性能的指标。
import joblib               # 导入joblib库，它是一个比pickle更高效、更适合大型NumPy数组的Python对象序列化模块，常用于保存和加载Scikit-learn模型。
from xgboost import XGBRegressor # 从xgboost库导入XGBRegressor类，用于构建XGBoost回归模型。

# 加载数据集
data = pd.read_excel('大学生低碳生活行为的影响因素数据集.xlsx')
# 使用pandas的read_excel函数，读取名为 '大学生低碳生活行为的影响因素数据集.xlsx' 的Excel文件到DataFrame `data` 中。

# 显示数据集的前五行
print(data.head())          # 打印DataFrame `data` 的前5行，帮助用户快速预览数据。

# 删除不必要的列并处理分类变量
data_cleaned = data.drop(columns=['序号', '所用时间'])  # 删除不必要的列
# 从DataFrame `data` 中删除 '序号'（通常是无用标识符）和 '所用时间'（可能与预测目标关系不大或存在数据质量问题）。
# `drop(columns=...)` 明确指定删除列。剩余的列赋予 `data_cleaned`。
data_cleaned = pd.get_dummies(data_cleaned, drop_first=True)  # 将分类变量转换为哑变量/指示变量
# `pd.get_dummies(data_cleaned, drop_first=True)` 对 `data_cleaned` 中所有具有对象类型（通常是字符串）的列进行独热编码。
# `drop_first=True` 是一个重要参数，它会为每个分类特征删除第一个类别对应的哑变量。
# 这可以避免多重共线性问题（即一个类别可以通过其他类别表示出来），对于线性模型尤其重要。

# 定义目标变量和特征
target = '5.您进行过绿色低碳的相关生活方式吗?'  # 确保这是目标变量
# 定义字符串 `target`，它存储了目标变量的列名。

# 定义自变量和因变量
X = data_cleaned.drop(columns=[target]) # 从 `data_cleaned` 中删除目标变量列，剩余的列作为特征（自变量），赋值给 `X`。
y = data_cleaned[target] # 选取 `target` 列作为目标变量（因变量），赋值给 `y`。

# 将数据拆分为训练集和测试集（测试集占20%）
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2, random_state=42)
# 使用train_test_split函数将数据集划分为训练集和测试集。
# `X`, `y`: 要分割的特征和目标变量。
# `test_size=0.2`: 测试集占总数据的20%。
# `random_state=42`: 随机种子，确保每次运行代码时分割结果一致。

# 初始化线性回归模型
model = LinearRegression() # 创建一个 `LinearRegression`（线性回归）模型实例。
# 训练线性回归模型
model.fit(X_train,y_train) # 使用训练集特征 `X_train` 和训练集目标 `y_train` 训练线性回归模型。

# 保存训练好的模型
model_filename = '2.2.4_model.pkl' # 定义保存模型的文件名。
joblib.dump(model,model_filename) # 使用 `joblib.dump()` 将训练好的 `model` 对象序列化并保存到指定文件。
                                  # joblib在此处替代pickle，因为它对于包含大量NumPy数组（Scikit-learn模型内部常用）的对象有优化。

# 进行预测
y_pred = model.predict(X_test) # 使用训练好的 `model` 对测试集特征 `X_test` 进行回归预测。

# 将结果保存到文本文件中
results = pd.DataFrame({'实际值': y_test, '预测值': y_pred})
# 创建一个pandas DataFrame，包含两列：'实际值'（来自 `y_test`）和 '预测值'（来自 `y_pred`）。
results_filename = '2.2.4_results.txt' # 定义保存结果的文件名。
results.to_csv(results_filename, index=False, sep='\t')  # 使用制表符分隔值保存到文本文件
# 将结果DataFrame保存为CSV文件 (`.txt` 扩展名但内容是CSV格式，`sep='\t'` 指定使用制表符作为分隔符)。
# `index=False` 表示在保存时不要写入DataFrame的索引。

# 将测试结果保存到报告文件中
report_filename = '2.2.4_report.txt' # 定义保存报告的文件名。
with open(report_filename, 'w') as f: # 以写入模式 ('w') 打开一个文本文件，用于保存报告。
    f.write(f'均方误差: {mean_squared_error(y_test,y_pred)}\n') # 写入均方误差 (MSE)。
    f.write(f'决定系数: {r2_score(y_test,y_pred)}\n')           # 写入决定系数 (R²)。
    
# 分析并纠正错误（示例：使用XGBoost）
# 初始化XGBoost模型（设定树的数量为1000，学习率为0.05，每棵树的最大深度为5，）
xgb_model = XGBRegressor(n_estimators=1000, learning_rate =0.05,max_depth=5, subsample=0.8, colsample_bytree=0.8)
# 创建一个 `XGBRegressor`（XGBoost回归）模型实例，并设置多个超参数：
# `n_estimators=1000`: 构建1000棵树。
# `learning_rate=0.05`: 学习率，每次迭代（构建一棵树）的权重收缩系数，控制每棵树对最终预测的贡献，防止过拟合。
# `max_depth=5`: 每棵树的最大深度，限制树的复杂度。
# `subsample=0.8`: 每次迭代随机采样80%的训练样本（行采样），进一步防止过拟合。
# `colsample_bytree=0.8`: 每次迭代随机采样80%的特征（列采样），防止过拟合。
# 训练XGBoost模型
xgb_model.fit(X_train,y_train) # 使用训练集特征 `X_train` 和训练集目标 `y_train` 训练XGBoost回归模型。

# 使用XGBoost模型进行预测
y_pred_xg = xgb_model.predict(X_test) # 使用训练好的 `xgb_model` 对测试集特征 `X_test` 进行回归预测。

# 将XGBoost结果保存到文本文件中
results_xg_filename = '2.2.4_results_xg.txt' # 定义保存XGBoost结果的文件名。
results_xg = pd.DataFrame({'实际值': y_test, '预测值': y_pred_xg}) # 创建DataFrame保存XGBoost的实际值和预测值。
results_xg.to_csv(results_xg_filename, index=False, sep='\t')  # 使用制表符分隔值保存到文本文件

# 将XGBoost测试结果保存到报告文件中
report_filename_xgb = '2.2.4_report_xgb.txt' # 定义保存XGBoost报告的文件名。
with open(report_filename_xgb, 'w') as f: # 打开文件保存XGBoost模型的报告。
    f.write(f'均方误差: {mean_squared_error(y_test,y_pred_xg)}\n') # 写入XGBoost模型的均方误差。
    f.write(f'决定系数: {r2_score(y_test,y_pred_xg)}\n') # 写入XGBoost模型的决定系数。
